# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be Sleeping Non-Astrocyte Expression values and Group 2 be SD Non-Astrocyte Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_NonAstro_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE69079.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 43)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 43)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM1692599_Astro_S.CEL',
       'GSM1692600_Astro_S.CEL', 'GSM1692601_Astro_S.CEL',
       'GSM1692602_Astro_S.CEL', 'GSM1692603_Astro_S.CEL',
       'GSM1692604_Astro_S.CEL', 'avg-Astro_S', 'log_fold-Astro_S_vs_Astro_SD',
       'fold-Astro_S_vs_Astro_SD', 'rawp-Astro_S_vs_Astro_SD',
       'adjp-Astro_S_vs_Astro_SD', 'GSM1692611_Astro_SD.CEL',
       'GSM1692612_Astro_SD.CEL', 'GSM1692613_Astro_SD.CEL',
       'GSM1692614_Astro_SD_.CEL', 'GSM1692615_Astro_SD.CEL',
       'GSM1692616_Astro_SD.CEL', 'avg-Astro_SD', 'GSM1692617_NonAstro_S.CEL',
       'GSM1692618_NonAstro_S.CEL', 'avg-NonAstro_S',
       'log_fold-NonAstro_S_vs_NonAstro_SD', 'fold-NonAstro_S_vs_NonAstro_SD',
       'rawp-NonAstro_S_vs_NonAstro_SD', 'adjp-NonAstro_S_vs_NonAstro

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='NonAstro_S.CEL'
sd_filt ='NonAstro_SD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM1692617_NonAstro_S.CEL,GSM1692618_NonAstro_S.CEL
Probesets,,
1427138_at,7.08990,7.19147
1425600_a_at,11.22009,11.34716
1457168_at,5.61581,4.07789
1450135_at,9.83430,9.94148
1424014_at,10.18211,10.09195


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM1692621_NonAstro_SD.CEL,GSM1692622_NonAstro_SD.CEL
Probesets,,
1427138_at,6.63031,7.24827
1425600_a_at,11.33963,11.39472
1457168_at,4.56403,4.42987
1450135_at,10.47329,9.88955
1424014_at,10.23977,10.22955


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.201395
1425600_a_at    0.083550
1457168_at     -0.349900
1450135_at      0.293530
1424014_at      0.097630
Name: 430AV2_NonAstro_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.643176
1425600_a_at    1.206516
1457168_at     -0.453309
1450135_at      0.989152
1424014_at      2.151924
Name: 430AV2_NonAstro_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.482382
1425600_a_at    0.904887
1457168_at     -0.339981
1450135_at      0.741864
1424014_at      1.613943
Name: 430AV2_NonAstro_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      1.051709
1425600_a_at    1.181960
1457168_at      1.025686
1450135_at      1.122303
1424014_at      1.578847
Name: 430AV2_NonAstro_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_NonAstro_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_NonAstro_Enrich,430AV2_NonAstro_poolStDev,430AV2_NonAstro_Cohens_d,430AV2_NonAstro_J,430AV2_NonAstro_Hedges_g,430AV2_NonAstro_Var_d,430AV2_NonAstro_Var_g,430AV2_NonAstro_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1434403_at,Spred2,"sprouty-related, EVH1 domain containing 2",ENSMUSG00000045671,114716,NaN,inactivation of MAPK activity // multicellular...,protein binding // stem cell factor receptor b...,cytoplasmic vesicle // cytoplasm // plasma mem...,Kit Receptor Signaling Pathway:WP407(WikiPathw...,"mmu-let-7d(pictar), mmu-miR-106a(miRanda), mmu...",...,0.002022,0.602080,0.602080,0.002562,235.042293,0.75,176.281720,6906.609949,3884.968096,62.329512
1421320_a_at,Qtrtd1,queuine tRNA-ribosyltransferase domain contain...,ENSMUSG00000022704,106248,NaN,tRNA modification // queuosine biosynthetic pr...,transferase activity // queuine tRNA-ribosyltr...,NaN,NaN,"mmu-miR-125a-5p(TargetScan|miRanda), mmu-miR-1...",...,0.119466,1.031055,0.610305,0.004905,124.420939,0.75,93.315704,1936.071268,1089.040088,33.000607
1432414_at,OTTMUSG00000015282,predicted gene 13999,ENSMUSG00000087458|,100036539,NaN,NaN,NaN,NaN,NaN,NaN,...,0.373886,0.620773,0.515800,0.005617,91.832801,0.75,68.874601,1055.157917,593.526328,24.362396
1436583_at,Dscc1,defective in sister chromatid cohesion 1 homol...,ENSMUSG00000022422,72107,NaN,DNA replication // cell cycle,DNA binding,nucleus,NaN,"mmu-miR-186(RNAhybrid|TargetScan|miRanda), mmu...",...,0.361433,0.311245,0.311245,0.003446,90.316825,0.75,67.737618,1020.641102,574.110620,23.960606
1428853_at,Ptch1,patched homolog 1,ENSMUSG00000021466,19206,NaN,dorsal/ventral pattern formation // smoothened...,cholesterol binding // protein binding // zinc...,extracellular region // integral to plasma mem...,Hedgehog Signaling Pathway:WP116(WikiPathways)...,"mmu-miR-101(TargetScan|pictar), mmu-miR-101ab(...",...,0.000067,1.967913,0.384265,0.004765,80.637195,0.75,60.477896,813.794644,457.759487,21.395315
1435357_at,D4Wsu53e,"DNA segment, Chr 4, Wayne State University 53,...",ENSMUSG00000037266,27981,NaN,NaN,NaN,NaN,NaN,"mmu-miR-106a(miRanda), mmu-miR-106b(miRanda), ...",...,0.000017,2.175512,0.805590,0.011595,69.477652,0.75,52.108239,604.393009,339.971068,18.438304
1419202_at,Cst7,cystatin F (leukocystatin),ENSMUSG00000068129,13011,NaN,NaN,endopeptidase inhibitor activity // cysteine p...,extracellular region,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,0.490582,0.257357,0.169765,0.002595,65.420039,0.75,49.065029,535.972680,301.484633,17.363313
1425031_at,Fktn,fukutin,ENSMUSG00000028414,246179,NaN,NaN,transferase activity,Golgi membrane // Golgi apparatus // integral ...,XPodNet - protein-protein interactions in the ...,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,0.001368,0.787558,0.753070,0.011753,64.072039,0.75,48.054030,514.153279,289.211219,17.006211
1435792_at,EG665317,component of Sp100-rs [Source:MGI Symbol;Acc:M...,ENSMUSG00000090186|ENSMUSG00000062783|ENSMUSG0...,665317,NaN,NaN,NaN,NaN,NaN,"mmu-miR-1(miRanda), mmu-miR-100(miRanda), mmu-...",...,0.225165,0.438035,0.131675,0.002083,63.208753,0.75,47.406565,500.418315,281.485302,16.777524


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM1692599_Astro_S.CEL',
       'GSM1692600_Astro_S.CEL', 'GSM1692601_Astro_S.CEL',
       'GSM1692602_Astro_S.CEL', 'GSM1692603_Astro_S.CEL',
       'GSM1692604_Astro_S.CEL', 'avg-Astro_S', 'log_fold-Astro_S_vs_Astro_SD',
       'fold-Astro_S_vs_Astro_SD', 'rawp-Astro_S_vs_Astro_SD',
       'adjp-Astro_S_vs_Astro_SD', 'GSM1692611_Astro_SD.CEL',
       'GSM1692612_Astro_SD.CEL', 'GSM1692613_Astro_SD.CEL',
       'GSM1692614_Astro_SD_.CEL', 'GSM1692615_Astro_SD.CEL',
       'GSM1692616_Astro_SD.CEL', 'avg-Astro_SD', 'GSM1692617_NonAstro_S.CEL',
       'GSM1692618_NonAstro_S.CEL', 'avg-NonAstro_S',
       'log_fold-NonAstro_S_vs_NonAstro_SD', 'fold-NonAstro_S_vs_NonAstro_SD',
       'rawp-NonAstro_S_vs_NonAstro_SD', 'adjp-NonAstro_S_vs_NonAstro

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM1692599_Astro_S.CEL,GSM1692600_Astro_S.CEL,GSM1692601_Astro_S.CEL,GSM1692602_Astro_S.CEL,GSM1692603_Astro_S.CEL,GSM1692604_Astro_S.CEL,avg-Astro_S,log_fold-Astro_S_vs_Astro_SD,fold-Astro_S_vs_Astro_SD,rawp-Astro_S_vs_Astro_SD,...,ANOVA-adjp,largest fold,430AV2_NonAstro_Enrich,430AV2_NonAstro_poolStDev,430AV2_NonAstro_Cohens_d,430AV2_NonAstro_J,430AV2_NonAstro_Hedges_g,430AV2_NonAstro_Var_d,430AV2_NonAstro_Var_g,430AV2_NonAstro_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_NonAstro_Enrich',u'430AV2_NonAstro_Hedges_g', u'430AV2_NonAstro_Var_g', u'430AV2_NonAstro_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_NonAstro_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_NonAstro_Enrich,430AV2_NonAstro_Hedges_g,430AV2_NonAstro_Var_g,430AV2_NonAstro_SEg
MGI symbol,,,,
0610005C13Rik,0.107075,1.139643,0.724848,0.851380
0610008F07Rik,-0.331460,-0.840505,0.650806,0.806726
0610009B22Rik,0.100225,0.417296,0.584267,0.764374
0610009D07Rik,-0.053253,-0.102271,0.565311,0.751869
0610009O20Rik,0.416420,1.651869,0.903584,0.950570
0610010K14Rik,-0.306825,-0.817068,0.645950,0.803710
0610012G03Rik,0.040325,0.036667,0.595764,0.771707
0610031J06Rik,-0.289480,-0.689129,0.621862,0.788583
0610037L13Rik,-0.195510,-0.860408,0.655038,0.809344
